# 🚀 SimpleKeyboardKotlin - Google Colab Build

Сборка Android APK в Google Colab с полной настройкой среды.

In [ ]:
# 1. Установка Java 17
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java

import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-17-openjdk-amd64'
os.environ['PATH'] = f"{os.environ['JAVA_HOME']}/bin:{os.environ['PATH']}"

!java -version

In [ ]:
# 2. Установка Android SDK
!mkdir -p /content/android-sdk/cmdline-tools
!cd /content/android-sdk/cmdline-tools && wget -q https://dl.google.com/android/repository/commandlinetools-linux-9477386_latest.zip
!cd /content/android-sdk/cmdline-tools && unzip -q commandlinetools-linux-9477386_latest.zip
!cd /content/android-sdk/cmdline-tools && mv cmdline-tools latest

os.environ['ANDROID_SDK_ROOT'] = '/content/android-sdk'
os.environ['ANDROID_HOME'] = '/content/android-sdk'
os.environ['PATH'] = f"{os.environ['ANDROID_SDK_ROOT']}/cmdline-tools/latest/bin:{os.environ['PATH']}"

print("Android SDK установлен")

In [ ]:
# 3. Установка компонентов Android SDK
!yes | sdkmanager --licenses
!sdkmanager "platforms;android-34"
!sdkmanager "build-tools;34.0.0"
!sdkmanager "platform-tools"

!sdkmanager --list_installed

In [ ]:
# 4. Клонирование проекта
!git clone https://github.com/Bokvlad01gmailcom/SimpleKeyboardKotlin.git
!ls -la SimpleKeyboardKotlin/

In [ ]:
# 5. Сборка APK
%cd SimpleKeyboardKotlin

os.environ['GRADLE_OPTS'] = '-Xmx2048m -Dfile.encoding=UTF-8'

!chmod +x gradlew
!./gradlew --version
!./gradlew clean assembleDebug --no-daemon --stacktrace

In [ ]:
# 6. Проверка результата
!find . -name "*.apk" -type f -exec ls -la {} \;

# Скачивание APK
from google.colab import files
import os

apk_path = 'app/build/outputs/apk/debug/app-debug.apk'
if os.path.exists(apk_path):
    print(f"✅ APK найден: {apk_path}")
    !ls -la {apk_path}
    files.download(apk_path)
else:
    print("❌ APK не найден")
    !find . -name "*.apk" -type f

In [ ]:
# 7. Создание подписанного APK (если нужно)
!keytool -genkey -v -keystore debug.keystore -alias androiddebugkey -keyalg RSA -keysize 2048 -validity 10000 -storepass android -keypass android -dname "CN=Android Debug,O=Android,C=US"

# Подписание APK
!jarsigner -verbose -sigalg SHA1withRSA -digestalg SHA1 -keystore debug.keystore -storepass android -keypass android app/build/outputs/apk/debug/app-debug.apk androiddebugkey

print("APK подписан для установки")